# Constant Bidder

## Environment

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

## Data Loader

In [3]:
data_path = os.path.abspath(os.pardir + '/Data/train.csv')
df = pd.read_csv(data_path, na_values=['Na', 'null']).fillna(0)

### Data Exploration

In [4]:
df_clicks = df.loc[df["click"] == 1]
df_clicks.reset_index(drop=True, inplace=True)

feature = 'advertiser'
df_grouped_mean = df_clicks.groupby([feature]).mean()
df_grouped_sum = df_clicks.groupby([feature]).sum()

df_grouped_mean['eCPC'] = df_grouped_sum['payprice'] / (df_grouped_sum['click'])
df_grouped_mean['ctr'] = df_grouped_sum['click'] / df['advertiser'].value_counts()

#print(df_grouped['eCPC'], "\n")

df_clicks.mean()
#df_grouped_mean

df_overview = pd.DataFrame(df_grouped_mean, columns=['ctr', 'eCPC', 'payprice', 'bidprice'])
print(df_overview)

                 ctr        eCPC    payprice    bidprice
advertiser                                              
1458        0.000782   96.576623   96.576623  300.000000
2259        0.000322  107.604651  107.604651  290.441860
2261        0.000327  132.000000  132.000000  287.388889
2821        0.000620  119.839695  119.839695  290.755725
2997        0.004355   89.036866   89.036866  277.000000
3358        0.000762  110.871287  110.871287  232.188119
3386        0.000703  118.737500  118.737500  300.000000
3427        0.000675  103.533088  103.533088  235.047794
3476        0.000602  101.406417  101.406417  248.812834


### Incoming bids

target and feature variables

'target' : 'click',     # 'payprice', 'bidprice'   # select the target

'features': ['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 
'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'keypage', 'advertiser', 'usertag'], # select the data features


In [ ]:
budget = 6250
constant_bid = 280

## Evaluation Stats_____________

bids_won = 0
earned_clicks = 0
ctr = 0                  # bids_won / earned_clicks
total_paid = 0
cpc = 0                  # cost per click

print("starting budget:", budget, "\n")

for index, row in df.iterrows():
        
    if constant_bid > budget: # check if budget is sufficient for bidprice
        constant_bid = budget
        print("constant bid reduced to:", constant_bid, ", total_paid:", total_paid, ", bids_won:", bids_won, "\n")
        
    # WON BID ______________________________________________

    if constant_bid >= row['bidprice']:     
        
        bids_won += 1                        # won the bid
        total_paid += row['payprice']        # add amount to total_paid   
        budget = budget - row['payprice']    # substract money from budget
        
        #if constant_bid == row['bidprice']:      
            #budget = budget - row['payprice']    # substract money from budget
            
        #elif constant_bid > row['bidprice']:
        #    budget = budget - row['bidprice']    # substract money from budget
                
        # CLICK = 1 ______________________________________________
    
        if row['click'] == 1:    # only reduce money from budget if ad has been clicked
            
                earned_clicks += 1                   # earn the click
                print("current budget:", budget, ", earned clicks:", earned_clicks, "\n")
                
        print("WON - bid#", index, ", budget:", budget, ", payprice:", row['payprice'], ", bids_won:", bids_won, ", earned_clicks:", earned_clicks, "\n")
        
    #if index%100000 == 0:
    #    print("bid#", index, ", budget:", budget, ", bids_won:", bids_won, ", earned_clicks:", earned_clicks, "\n")

        
print("__________________________________\n")

if earned_clicks > 0:
    cpc = total_paid / earned_clicks
if bids_won > 0:
    ctr = earned_clicks / bids_won

print("left budget:", budget)
print("earned clicks:", earned_clicks)
print("CTR:", ctr)
print("cost per click:", cpc)
        